# Serve Recommendations Notebook

This notebook mirrors `src/inference/serve_recommendations.py` and lets you:

- Load the trained two-tower SBERT model and product corpus.
- Run ad-hoc queries to get product recommendations (embedding-based: same encoder for query and products; top-k by **cosine similarity**—no text generation).
- Optionally pull queries from `processed/.../eval_queries.json` by ID.

**Embedding index:** Product embeddings are cached on disk under `<corpus_parent>/.embedding_index/` (per model + corpus) so they are not recomputed every run. Use `use_index=False` in `load_recommender()` to disable loading/saving the cache.

Adjust paths as needed if you move the project or models.

In [1]:
from pathlib import Path
import json
import logging

from src.constants import DEFAULT_CORPUS_PATH, DEFAULT_MODEL_DIR, PROJECT_ROOT
from src.inference.serve_recommendations import load_recommender

logging.basicConfig(level=logging.INFO, format="%(message)s")

PROJECT_ROOT, DEFAULT_MODEL_DIR, DEFAULT_CORPUS_PATH

(PosixPath('/Users/chen_bowen/AI & ML/Projects/Instacart_Personalization'),
 PosixPath('/Users/chen_bowen/AI & ML/Projects/Instacart_Personalization/models/two_tower_sbert/final'),
 PosixPath('/Users/chen_bowen/AI & ML/Projects/Instacart_Personalization/processed/eval_corpus.json'))

In [2]:
# Configure paths (override these if you have a different model or corpus location)
model_dir = DEFAULT_MODEL_DIR
corpus_path = DEFAULT_CORPUS_PATH

print("Model dir:", model_dir)
print("Corpus path:", corpus_path)

# use_index=True (default): load/save product embeddings in .embedding_index/ to avoid recomputing each run
rec = load_recommender(
    model_dir=model_dir,
    corpus_path=corpus_path,
    use_index=True,  # set False to recompute product embeddings every time
)
rec

Use pytorch device_name: mps
Load pretrained SentenceTransformer: /Users/chen_bowen/AI & ML/Projects/Instacart_Personalization/models/two_tower_sbert/final


Model dir: /Users/chen_bowen/AI & ML/Projects/Instacart_Personalization/models/two_tower_sbert/final
Corpus path: /Users/chen_bowen/AI & ML/Projects/Instacart_Personalization/processed/eval_corpus.json


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

Batches:   0%|          | 0/777 [00:00<?, ?it/s]

Loaded model from /Users/chen_bowen/AI & ML/Projects/Instacart_Personalization/models/two_tower_sbert/final, corpus 49688 products from /Users/chen_bowen/AI & ML/Projects/Instacart_Personalization/processed/eval_corpus.json


In [ ]:
def recommend_notebook(
    query: str,
    top_k: int = 10,
    exclude_product_ids: set[str] | None = None,
):
    """Helper to get and pretty-print recommendations (top-k by cosine similarity)."""
    results = rec.recommend(query=query, top_k=top_k, exclude_product_ids=exclude_product_ids)
    print(f"Top-{top_k} recommendations:")
    for i, (pid, score) in enumerate(results, 1):
        text = rec.pid_to_text[pid]
        print(f"  {i}. product_id={pid} (score={score:.4f}) {text[:60]}...")
    return results

# Example demo query (same as in src.inference.serve_recommendations.main)
demo_query = "[+7d w4h14] Organic Milk, Whole Wheat Bread."
results = recommend_notebook(demo_query, top_k=10)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
# Optional: use an eval query from processed/eval_queries.json by ID
eval_queries_path = corpus_path.parent / "eval_queries.json"
print("Eval queries path:", eval_queries_path)

if eval_queries_path.exists():
    with open(eval_queries_path, "r") as f:
        eval_queries = json.load(f)
    print(f"Loaded {len(eval_queries)} eval queries")

    # Pick an ID to inspect
    sample_id = next(iter(eval_queries.keys()))
    query = eval_queries[sample_id]
    print(f"Sample eval_query_id={sample_id}\n{query}\n")

    _ = recommend_notebook(query, top_k=10)
else:
    print("No eval_queries.json found next to corpus; skip eval-query-based example.")

Eval queries path: /Users/chen_bowen/AI & ML/Projects/Instacart_Personalization/processed/eval_queries.json
Loaded 13120 eval queries
Sample eval_query_id=3178496
Previously ordered: [ordered on weekday 0 at hour 13.0] Chicken Biryani, Bibimbop With Beef & Brown Rice, Beef Bulgogi With Brown Rice, World Cuisine Certified Halal Chicken Tikka Masala With Basmati Rice, Lamb Vindaloo w/ Basmati Rice, Italian Herbs & Olive Oil Mozzarella Stix, 98% fatfree smart classics roasted Tri-Cut Potatoes with sea salt, Onion Rings, Spinach Dip, Tater Puffs; [ordered 6 days after previous order on weekday 6 at hour 16.0] Spinach Dip, Palak (Saag) Paneer, Light & Lean Black Bean and Cheese Enchilada, Amy Indian Mattar Paneer Light In Sodium, Lemongrass Basil Chicken With Jasmine Rice, Organic Jasmine Rice, Chile Verde Chicken, Orange Mango Chicken, Tandoori Inspired Spiced Chicken, Braised Beef & Polenta Red Wine; [ordered 5 days after previous order on weekday 4 at hour 16.0] Spinach Dip, Tandoori Ins

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Top-10 recommendations:
  1. product_id=47364 (score=0.5529) Product: Lamb Vindaloo w/ Basmati Rice. Aisle: frozen meals....
  2. product_id=11314 (score=0.5317) Product: Chicken Vindaloo with Basmati Rice. Aisle: frozen m...
  3. product_id=40521 (score=0.4849) Product: Paneer Tikka Masala With Rajma & Onion Basmati Rice...
  4. product_id=6975 (score=0.4832) Product: World Cuisine Certified Halal Chicken Tikka Masala ...
  5. product_id=9681 (score=0.4771) Product: Chicken Tikka Masala with Cumin Infused Basmati Ric...
  6. product_id=36766 (score=0.4750) Product: Chicken Biryani. Aisle: frozen meals. Department: f...
  7. product_id=4042 (score=0.4734) Product: Chicken Curry with Seasoned Basmati Rice. Aisle: fr...
  8. product_id=28879 (score=0.4722) Product: Amy Indian Mattar Paneer Light In Sodium. Aisle: fr...
  9. product_id=46642 (score=0.4642) Product: Mild Heat Level Lamb Saag with Basmati Rice. Aisle:...
  10. product_id=3428 (score=0.4560) Product: Skillet Meals For 2 Chic